In [1]:
!mkdir models
import requests

# files = ['regression.py','sindy_helper.py','dynamicalsystems.py','fullSINDyAutoencoder.py']
# for name in files:
#     with requests.get('https://raw.githubusercontent.com/audrey163/Autoencoder/main/'+name, stream=True) as r:
#         r.raise_for_status()
#         with open(name, 'wb') as f:
#             for chunk in r.iter_content(chunk_size=8192): 
#                 f.write(chunk)
import numpy as np
import pprint as pp
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import math
from scipy.stats import special_ortho_group
from scipy.integrate import solve_ivp
from regression import PolynomialLibrary, TrigLibrary
import sindy_helper
from dynamicalsystems import SimplePendulum
from fullSINDyAutoencoder import FullSINDyAutoencoder

#if torch.cuda.is_available():
#    device = torch.device('cuda:0')
#    print("GPU")
#else:
#   print("CPU")
device = torch.device('cpu')

mkdir: cannot create directory ‘models’: File exists


In [2]:
dataset = SimplePendulum(g=9.8,l=2,mu=0.5,theta0=np.pi/2)
total_samples = len(dataset)

params = {
    'architecture' : { 
        'input_dimension' : 10,
        'latent_dimension' : 2 ,
        'SIND-y' : {}
    },
    'optimization' : { 
        'learn_rate' : 1e-2,
        'loss_reg' : {
            'X' : 1,
            'SINDy' : 1,
            'dX' : 1, #dX is regularization is 0 because dX_pred = None
            'Xi1' : 5,
            'Xi2' : 5
        }
    },
    'training' : {
        'epochs' : 500,
        'batch_size' : 500,
        'save_freq' : 5,
        'load_weghts_from' : ':latest' # 'random.init', ':latest', None, 'models/FILENAME'
    },
    'inputs' : {
        'total_samples' : total_samples,
    },
    'outputs' : {
        'loss' : True,
        'sindy_show' : False,
    }
    
}

dataset.to(device)
dataloader = DataLoader(dataset=dataset, batch_size=params['training']['batch_size'], shuffle=True, num_workers=2)
dataiter = iter(dataloader)

def load_model(state_dict_PATH=None):
    model = FullSINDyAutoencoder(params)
    if state_dict_PATH is not None:
        if state_dict_PATH == ':latest':
            from os import listdir
            from os.path import isfile, join
            onlyfiles = [f for f in listdir('models/') if isfile(join('models/', f))]
            onlyfiles.sort(reverse=True)
            state_dict_PATH = 'models/' + onlyfiles[0]
        model.load_state_dict(torch.load(state_dict_PATH))
        print("Using Model: " + state_dict_PATH)
    return model

#model = load_model(state_dict_PATH =':latest')
model = load_model()
model.to(device)
#
x = torch.tensor([1,2,3])
x.to(device)

tensor([1, 2, 3])

In [ ]:
model.train(dataset,dataloader,params)

Epoch 0
	Loss: {'X': 83.5987548828125, 'dZ': 11.383085250854492, 'dX': 170.66781616210938, 'Xi1': 1.18890380859375, 'Xi2': 1.5188647508621216}
Epoch 1
	Loss: {'X': 74.96044158935547, 'dZ': 35.47883987426758, 'dX': 174.59469604492188, 'Xi1': 0.3954358398914337, 'Xi2': 0.49556881189346313}
Epoch 2
